### Clean Code

#### Create riotwatcher object

In [22]:
import riotwatcher
import pandas as pd

In [23]:
def get_api_key() -> str:
    with open("../api_key.txt", "r") as file:
        return file.read()


API_KEY = get_api_key()

lolwatcher = riotwatcher.LolWatcher(api_key=API_KEY)

#### Create Queue Enum

In [24]:
from enum import Enum


class Queue(Enum):
    RANKED = 420
    NORMAL = 400
    ARAM = 450

#### Create Server / Region dict

In [25]:
regions = {
    "EUROPE": "EUW1",
    "ASIA": "KR",
    "AMERICAS": "NA"
}

#### Constants

In [26]:
MAX_GAME_COUNT = 100

#### Input Values

In [27]:
summoner_name = "TRM%20Maxi%20King"
server = "EUW1"
region = list(regions.keys())[list(regions.values()).index(server)]
# number of games
number_of_games = 2000
# till_patch = "13.15"
queue = Queue.ARAM

#### Relevant Game Keys

* metadata keys

In [28]:
META_DATA_KEYS = ["matchId"]

* info data keys

In [29]:
INFO_DATA_KEYS = ["gameCreation", "gameDuration", "gameVersion"]

* player data keys

In [30]:
PLAYER_DATA_KEYS = ["kills", "deaths", "assists", "championName", "teamId", "win"]

* all participant info

In [31]:
ALL_PARTICIPANT_KEYS = ["opponent"]

#### Get match list

In [32]:
def get_match_list(region: str, puuid: str, number_of_games: int, queue: Queue):
    matchlist = []

    count = 100 if number_of_games > 100 else number_of_games

    for i in range((number_of_games // MAX_GAME_COUNT) + 1):
        current_len = len(matchlist)
        matchlist.extend(lolwatcher.match.matchlist_by_puuid(region=region, puuid=puuid, start=i*MAX_GAME_COUNT, count=count, queue=queue.value))
        # Break when no games where added by latest matchlist_by_puuid call
        if current_len == len(matchlist):
            break

    return matchlist

#### Get Matchdata

In [33]:
def get_match_data(puuid: str, region: str, number_of_games: int, queue: Queue):
    try:
        matchlist = get_match_list(region=region, puuid=puuid, number_of_games=number_of_games, queue=queue)

        for match_id in matchlist:
            yield lolwatcher.match.by_id(region=region, match_id=match_id)

    except riotwatcher.ApiError as err:
            if err.response.status_code == 429:
                print('We should retry in {} seconds.'.format(err.headers['Retry-After']))
            else:
                print(err)
                raise 

#### Get participant player data

In [34]:
def get_participant_game_data(match_data: dict[str, str], puuid: str) -> dict[str, str]:
    participant_index = match_data["metadata"]["participants"].index(puuid)
    return match_data["info"]["participants"][participant_index]

#### Get opponent player data

In [35]:
def get_opponent_player_data(match_data: dict[str, str], puuid: str) -> dict[str, str]:
    participant_index = match_data["metadata"]["participants"].index(puuid)
    player_team_position = match_data["info"]["participants"][participant_index]["teamPosition"]
    for player_data in match_data["info"]["participants"]:

        if player_data["teamPosition"] == player_team_position and (player_data["puuid"] != puuid):
            return {"opponentChampion": player_data["championName"]}

#### Extract relevant match data

* extract data

In [36]:
def extract_data(data: dict[str, str], key_list: list[str]):
    match_data = {}
    for key in key_list:
        match_data[key] = data[key]
    return match_data

* extract match data

In [37]:
def extract_match_data(match_data: dict[str, str], puuid: str) -> dict[str, str]:
    game_data = {}
    game_data.update(extract_data(data=match_data["metadata"], key_list=META_DATA_KEYS))
    game_data.update(extract_data(data=match_data["info"], key_list=INFO_DATA_KEYS))
    game_data.update(extract_data(data=get_participant_game_data(match_data, puuid), key_list=PLAYER_DATA_KEYS))
    # game_data.update(get_opponent_player_data(match_data=match_data, puuid=puuid))
    return game_data

#### Load data in dataframe

In [38]:
puuid = lolwatcher.summoner.by_name(region=server, summoner_name=summoner_name)["puuid"]

match_data_generator = get_match_data(puuid=puuid, region=region, number_of_games=number_of_games, queue=queue)

player_data = []

for data in match_data_generator:
    match_data = extract_match_data(data, puuid)
    patch = ".".join(match_data["gameVersion"].split(".")[:2])
    # if patch == till_patch:
    #     break
    player_data.append(match_data)

df = pd.DataFrame(player_data)

df

,matchId,gameCreation,gameDuration,gameVersion,kills,deaths,assists,championName,teamId,win
0,EUW1_6672630593,1699745725751,1431,13.22.541.9804,6,12,33,Azir,200,False
1,EUW1_6672613086,1699745219333,111,13.22.541.9804,0,0,0,Jinx,100,False
2,EUW1_6672582767,1699743739869,1047,13.22.541.9804,14,8,22,KogMaw,200,True
3,EUW1_6672536147,1699742177261,1313,13.22.541.9804,11,8,30,Yasuo,200,True
4,EUW1_6653859691,1698596172593,1423,13.21.539.4823,16,12,37,Xerath,200,False
...,...,...,...,...,...,...,...,...,...,...
534,EUW1_5872829983,1652621248000,1905,12.9.439.127,32,13,25,Zed,100,True
535,EUW1_5872815327,1652619432000,1560,12.9.439.127,9,10,26,Ashe,200,True
536,EUW1_5870294390,1652479852000,1249,12.9.439.127,7,10,19,Lux,100,True
537,EUW1_5870116659,1652474376000,1577,12.9.439.127,10,8,26,Velkoz,200,True


#### Clean up data

* convert unix timestamp in milliseconds to datetime

In [39]:
import datetime

def convert_unix_to_datetime(timestamp_ms):
    # Convert milliseconds to seconds
    timestamp_sec = timestamp_ms / 1000.0
    
    # Create a datetime object
    datetime_obj = datetime.datetime.utcfromtimestamp(timestamp_sec).replace(microsecond=0)
    
    return datetime_obj

In [40]:
# Convert TimestampColumn to datetime and round down to the nearest second
df["gameCreation"] = pd.to_datetime(df["gameCreation"], unit='ms').dt.floor('s')
df

,matchId,gameCreation,gameDuration,gameVersion,kills,deaths,assists,championName,teamId,win
0,EUW1_6672630593,2023-11-11 23:35:25,1431,13.22.541.9804,6,12,33,Azir,200,False
1,EUW1_6672613086,2023-11-11 23:26:59,111,13.22.541.9804,0,0,0,Jinx,100,False
2,EUW1_6672582767,2023-11-11 23:02:19,1047,13.22.541.9804,14,8,22,KogMaw,200,True
3,EUW1_6672536147,2023-11-11 22:36:17,1313,13.22.541.9804,11,8,30,Yasuo,200,True
4,EUW1_6653859691,2023-10-29 16:16:12,1423,13.21.539.4823,16,12,37,Xerath,200,False
...,...,...,...,...,...,...,...,...,...,...
534,EUW1_5872829983,2022-05-15 13:27:28,1905,12.9.439.127,32,13,25,Zed,100,True
535,EUW1_5872815327,2022-05-15 12:57:12,1560,12.9.439.127,9,10,26,Ashe,200,True
536,EUW1_5870294390,2022-05-13 22:10:52,1249,12.9.439.127,7,10,19,Lux,100,True
537,EUW1_5870116659,2022-05-13 20:39:36,1577,12.9.439.127,10,8,26,Velkoz,200,True


* convert gameVersion (13.22.541.9804) to patch (13.22)

In [41]:
# Anwendung eines Lambda-Ausdrucks auf die "gameVersion"-Spalte
df['gameVersion'] = df['gameVersion'].apply(lambda x: ".".join(x.split('.')[:2]))

# Ausgabe des aktualisierten DataFrames
df


,matchId,gameCreation,gameDuration,gameVersion,kills,deaths,assists,championName,teamId,win
0,EUW1_6672630593,2023-11-11 23:35:25,1431,13.22,6,12,33,Azir,200,False
1,EUW1_6672613086,2023-11-11 23:26:59,111,13.22,0,0,0,Jinx,100,False
2,EUW1_6672582767,2023-11-11 23:02:19,1047,13.22,14,8,22,KogMaw,200,True
3,EUW1_6672536147,2023-11-11 22:36:17,1313,13.22,11,8,30,Yasuo,200,True
4,EUW1_6653859691,2023-10-29 16:16:12,1423,13.21,16,12,37,Xerath,200,False
...,...,...,...,...,...,...,...,...,...,...
534,EUW1_5872829983,2022-05-15 13:27:28,1905,12.9,32,13,25,Zed,100,True
535,EUW1_5872815327,2022-05-15 12:57:12,1560,12.9,9,10,26,Ashe,200,True
536,EUW1_5870294390,2022-05-13 22:10:52,1249,12.9,7,10,19,Lux,100,True
537,EUW1_5870116659,2022-05-13 20:39:36,1577,12.9,10,8,26,Velkoz,200,True


* change teamPosition: UTILITY to SUPPORT

In [43]:
# df.loc[df['teamPosition'] == 'UTILITY', 'teamPosition'] = 'SUPPORT'
df

df.tail()

,matchId,gameCreation,gameDuration,gameVersion,kills,deaths,assists,championName,teamId,win
534,EUW1_5872829983,2022-05-15 13:27:28,1905,12.9,32,13,25,Zed,100,True
535,EUW1_5872815327,2022-05-15 12:57:12,1560,12.9,9,10,26,Ashe,200,True
536,EUW1_5870294390,2022-05-13 22:10:52,1249,12.9,7,10,19,Lux,100,True
537,EUW1_5870116659,2022-05-13 20:39:36,1577,12.9,10,8,26,Velkoz,200,True
538,EUW1_5860703912,2022-05-07 19:43:47,1416,12.8,18,5,24,Senna,200,True


In [54]:
siege_pro_champion = df[df["championName"] = "Kalye"]
siege_pro_champion

'Kalye'

#### Save dataframe to parquet

In [ ]:
import pyarrow

df.to_parquet("NowayEUW.parquet")

Load dataframe from parquet

In [ ]:
df = pd.read_parquet("NowayKR.parquet")

In [1]:
pd.set_option('display.max_rows', 10)
# pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

NameError: name 'pd' is not defined

#### Role percentage

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
games_count = df.shape[0]

role_percentage = df["teamPosition"].value_counts() / games_count * 100

# Erstellen des Balkendiagramms
role_percentage.plot(kind='bar', color='skyblue')

# Hinzufügen von Beschriftungen und Titel
plt.xlabel('Roles')
plt.ylabel('Percentage')
plt.title('Role Percentage Distribution')

# plt.savefig("NowayKRRolePercentage")
# Anzeigen des Diagramms
plt.show()


#### Role / Win Percentage

In [ ]:
role_counts = df["teamPosition"].value_counts()
wins_by_position = df.groupby('teamPosition')['win'].sum()

winrate_by_position = round((wins_by_position / role_counts) * 100, 1)
winrate_by_position

In [ ]:
# Erstellen des Balkendiagramms
winrate_by_position.plot(kind='bar', color='skyblue')

# Hinzufügen von Beschriftungen und Titel
plt.xlabel('Roles')
plt.ylabel('Winrate Percentage')
plt.title('Role Winrate Percentage Distribution')

# plt.savefig("NowayKRRolePercentage")
# Anzeigen des Diagramms
plt.show()
